In [25]:
import nltk
import re
import numpy as np # linear algebra
import pandas as pd # data processing
import random
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize


#import pandas_profiling

%matplotlib inline

In [2]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
raw_dataset=pd.read_csv('/content/drive/MyDrive/NLP project/Amazon_Unlocked_Mobile.csv')
raw_dataset.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [27]:
raw_dataset.tail()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
413835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
413836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
413837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
413838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0
413839,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,4,Only downside is that apparently Verizon no lo...,0.0


In [28]:
df=raw_dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Product Name  413840 non-null  object 
 1   Brand Name    348669 non-null  object 
 2   Price         407907 non-null  float64
 3   Rating        413840 non-null  int64  
 4   Reviews       413778 non-null  object 
 5   Review Votes  401544 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 18.9+ MB


In [29]:
df.shape

(413840, 6)

In [30]:
df.describe()

,Price,Rating,Review Votes
count,407907.000000,413840.000000,401544.000000
mean,226.867155,3.819578,1.507237
std,273.006259,1.548216,9.163853
min,1.730000,1.000000,0.000000
25%,79.990000,3.000000,0.000000
50%,144.710000,5.000000,0.000000
75%,269.990000,5.000000,1.000000
max,2598.000000,5.000000,645.000000


In [31]:
df=df[['Reviews','Rating']]

In [32]:
df.head()

,Reviews,Rating
0,I feel so LUCKY to have found this used (phone...,5
1,"nice phone, nice up grade from my pantach revu...",4
2,Very pleased,5
3,It works good but it goes slow sometimes but i...,4
4,Great phone to replace my lost phone. The only...,4


In [33]:
df.tail()

,Reviews,Rating
413835,another great deal great price,5
413836,Ok,3
413837,Passes every drop test onto porcelain tile!,5
413838,I returned it because it did not meet my needs...,3
413839,Only downside is that apparently Verizon no lo...,4


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  413778 non-null  object
 1   Rating   413840 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.3+ MB


In [35]:
df.isnull().sum() 

Reviews    62
Rating      0
dtype: int64

In [36]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413778 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  413778 non-null  object
 1   Rating   413778 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ MB


In [37]:
df.isnull()      
# Returns a boolean matrix, if the value is NaN then True otherwise False
df.isnull().sum() 
# Returns the column names along with the number of NaN values in that particular column

Reviews    0
Rating     0
dtype: int64

In [38]:
df=df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413778 entries, 0 to 413777
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  413778 non-null  object
 1   Rating   413778 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.3+ MB


In [39]:
import nltk

In [40]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
Cstopwords=set(stopwords.words('english')+list(punctuation))
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()
def clean_review(review_column):
    review_corpus=[]
    for i in range(0,len(review_column)):
        review=review_column[i]
        review=re.sub('[^a-zA-Z]',' ',review)
        review=str(review).lower()
        review=word_tokenize(review)
        review=[stemmer.stem(w) for w in review if w not in Cstopwords]
        review=[lemma.lemmatize(w) for w in review ]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [42]:
review_column=df['Reviews']
review_corpus=clean_review(review_column)

In [43]:
df['clean_review']=review_corpus
df.tail(20)

,Reviews,Rating,clean_review
413758,They said phone was normal wear but it was a l...,1,said phone normal wear lie phone complet scrat...
413759,"You could shoot this out of a potato gun, and ...",5,could shoot potato gun proceed dislodg tree or...
413760,Bought this for my mother and she loves it. Gr...,5,bought mother love great flip phone
413761,"Excellent phone, as advertised. Love the push-...",5,excel phone advertis love push talk featur
413762,works great and picks up signal in place my ot...,4,work great pick signal place phone
413763,"Great phone. Large keys, best flip phone I hav...",5,great phone larg key best flip phone own
413764,"Pros...Works great, very durable, easy to navi...",5,pro work great durabl easi navig speaker fanta...
413765,just as described perfect for the price,5,describ perfect price
413766,Would not work,1,would work
413767,"Speaker phone doesn't work, but phone works good",3,speaker phone work phone work good


In [44]:
!pip install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
#review summarization
def summarize_reviews(reviews):
  res=[]
  for text in reviews:
    parser = PlaintextParser.from_string(text,Tokenizer("english"))
    summarizer = LuhnSummarizer()
    summary =summarizer(parser.document,2)
    text_summary=""
    for sentence in summary:
      text_summary+=str(sentence)
    res.append(text_summary)
  return res

In [46]:
df['clean_summarized_review'] = summarize_reviews(df['clean_review'])
df.tail(20)

,Reviews,Rating,clean_review,clean_summarized_review
413758,They said phone was normal wear but it was a l...,1,said phone normal wear lie phone complet scrat...,said phone normal wear lie phone complet scrat...
413759,"You could shoot this out of a potato gun, and ...",5,could shoot potato gun proceed dislodg tree or...,could shoot potato gun proceed dislodg tree or...
413760,Bought this for my mother and she loves it. Gr...,5,bought mother love great flip phone,bought mother love great flip phone
413761,"Excellent phone, as advertised. Love the push-...",5,excel phone advertis love push talk featur,excel phone advertis love push talk featur
413762,works great and picks up signal in place my ot...,4,work great pick signal place phone,work great pick signal place phone
413763,"Great phone. Large keys, best flip phone I hav...",5,great phone larg key best flip phone own,great phone larg key best flip phone own
413764,"Pros...Works great, very durable, easy to navi...",5,pro work great durabl easi navig speaker fanta...,pro work great durabl easi navig speaker fanta...
413765,just as described perfect for the price,5,describ perfect price,describ perfect price
413766,Would not work,1,would work,would work
413767,"Speaker phone doesn't work, but phone works good",3,speaker phone work phone work good,speaker phone work phone work good


In [47]:
df['summarized_review'] = summarize_reviews(df['Reviews'])
df.tail(20)

,Reviews,Rating,clean_review,clean_summarized_review,summarized_review
413758,They said phone was normal wear but it was a l...,1,said phone normal wear lie phone complet scrat...,said phone normal wear lie phone complet scrat...,They said phone was normal wear but it was a l...
413759,"You could shoot this out of a potato gun, and ...",5,could shoot potato gun proceed dislodg tree or...,could shoot potato gun proceed dislodg tree or...,"You could shoot this out of a potato gun, and ..."
413760,Bought this for my mother and she loves it. Gr...,5,bought mother love great flip phone,bought mother love great flip phone,Bought this for my mother and she loves it.Gre...
413761,"Excellent phone, as advertised. Love the push-...",5,excel phone advertis love push talk featur,excel phone advertis love push talk featur,"Excellent phone, as advertised.Love the push-t..."
413762,works great and picks up signal in place my ot...,4,work great pick signal place phone,work great pick signal place phone,works great and picks up signal in place my ot...
413763,"Great phone. Large keys, best flip phone I hav...",5,great phone larg key best flip phone own,great phone larg key best flip phone own,"Great phone.Large keys, best flip phone I have..."
413764,"Pros...Works great, very durable, easy to navi...",5,pro work great durabl easi navig speaker fanta...,pro work great durabl easi navig speaker fanta...,"Pros...Works great, very durable, easy to navi..."
413765,just as described perfect for the price,5,describ perfect price,describ perfect price,just as described perfect for the price
413766,Would not work,1,would work,would work,Would not work
413767,"Speaker phone doesn't work, but phone works good",3,speaker phone work phone work good,speaker phone work phone work good,"Speaker phone doesn't work, but phone works good"


In [48]:
df['Reviews'][15]

'Had this phone before and loved it but was not working so I got this phone. One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer. The card is fine was able to open on another phone. Trying to trouble shoot it for now.'

In [49]:
df['summarized_review'][15]

'One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer.The card is fine was able to open on another phone.'